In [1]:
import pandas as pd
import pickle

In [3]:
# Import international HSR data
hsr_df = pd.read_csv('../data/international_train_data.csv')
hsr_df

,Origin,Destination,Distance_meters,Duration
0,Barcelona,Madrid,620,3h12m
1,Madrid,Sevilla,530,2h35m
2,Madrid,Valladolid,178,0h55m
3,Madrid,Valencia,362,1h56m
4,Beijing,Shanghai,1200,4h52m
5,Shanghai,Hangzhou,174,1h6m
6,Shanghai,Wuhan,800,5h0m
7,Paris,Lille,346,1h10m
8,Paris,Lyon,483,1h56m
9,Paris,Bordeaux,292,2h17m
